In [32]:
import pandas as pd
df = pd.read_csv('train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [33]:
# Check data for null values
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [34]:
# drop null values
df = df.dropna(axis=0)
df.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [35]:
# Get the independent columns
X = df.drop('label', axis=1)
X.shape

(18285, 4)

In [36]:
y = df['label']
y.shape

(18285,)

In [37]:
# Import Text preprocessing libraries
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rajdubey\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [38]:
sentences = X.copy()
sentences.reset_index(inplace=True)
sentences

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...,...
18280,20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
18281,20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
18282,20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
18283,20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [39]:
# Text Preprocessing
try:
    
    ps = PorterStemmer()
    corpus = []
    for i in range(0, len(sentences)):
        #print(sentences['title'][i])
        sentence = re.sub('[^a-zA-Z]', ' ', sentences['title'][i])
        sentence = sentence.lower()
        words = sentence.split()
        words = [ps.stem(word) for word in words if not word in stopwords.words('english')]
        sentence = ' '.join(words)
        corpus.append(sentence)
except Exception as e:
    print(f'exception occured {str(e)}')

In [40]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [41]:
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot

In [47]:
#Vocabulary_size
vocab_size = 100
# One hot representation
one_hot_repr = [one_hot(words, vocab_size) for words in corpus]
one_hot_repr

[[37, 37, 64, 95, 31, 21, 45, 76, 2, 49],
 [41, 29, 66, 98, 35, 51, 79],
 [34, 68, 85, 69],
 [17, 62, 67, 12, 50, 43],
 [14, 35, 64, 40, 68, 77, 35, 64, 69, 63],
 [8, 31, 97, 22, 95, 23, 32, 23, 92, 30, 5, 12, 78, 91, 79],
 [92, 67, 99, 16, 72, 37, 68, 19, 23, 81, 95],
 [59, 93, 92, 82, 26, 84, 23, 87, 23, 81, 95],
 [16, 75, 83, 52, 2, 57, 97, 92, 23, 82],
 [72, 21, 64, 49, 7, 97, 44, 3],
 [79, 70, 33, 36, 94, 77, 98, 36, 93, 71, 40],
 [12, 73, 31, 57, 23, 26],
 [12, 31, 20, 82, 33, 61, 97, 34, 61],
 [31, 31, 27, 47, 16, 69, 49, 23, 81, 95],
 [96, 48, 29, 29, 19, 23, 81, 95],
 [16, 3, 76, 87, 38, 38, 4, 85, 43, 4],
 [62, 87, 29],
 [51, 90, 17, 87, 23, 90, 59, 79],
 [54, 49, 66, 42, 11, 11, 73, 80, 77],
 [41, 39, 23, 55, 15, 79],
 [10, 25, 68, 15, 49, 73, 19, 30, 9, 23, 81, 95],
 [69, 43, 96, 6, 47, 54, 2],
 [78, 2, 52, 11, 57, 1, 81, 82, 4, 50, 92, 79],
 [44, 29, 39, 31, 1, 68, 26, 14],
 [92, 33, 75, 92, 80, 19, 42, 16, 94, 23, 81, 95],
 [29, 66, 18, 98, 23, 81, 95],
 [51, 90, 17, 38, 

In [50]:
#Embedded docs -> apply padding
embedded_docs = pad_sequences(one_hot_repr, padding='pre', maxlen=20)

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 37, 37, 64, 95, 31, 21, 45,
       76,  2, 49])

In [56]:
# Create model
embedded_docs_features = 40
model = Sequential()
model.add(Embedding(vocab_size, embedded_docs_features))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

print(model.summary())

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_4 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [58]:
# Prepare training data
import numpy as np
from sklearn.model_selection import train_test_split
X_final = np.array(embedded_docs)
y_final = np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=.3, random_state=40)
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=50)

Epoch 1/10
256/256 ━━━━━━━━━━━━━━━━━━━━ 6s 15ms/step - accuracy: 0.8045 - loss: 0.4344 - val_accuracy: 0.8901 - val_loss: 0.2714
Epoch 2/10
256/256 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.8861 - loss: 0.2707 - val_accuracy: 0.8932 - val_loss: 0.2617
Epoch 3/10
256/256 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.8929 - loss: 0.2524 - val_accuracy: 0.8925 - val_loss: 0.2592
Epoch 4/10
256/256 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.8976 - loss: 0.2447 - val_accuracy: 0.8812 - val_loss: 0.2770
Epoch 5/10
256/256 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step - accuracy: 0.8966 - loss: 0.2426 - val_accuracy: 0.8941 - val_loss: 0.2581
Epoch 6/10
256/256 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step - accuracy: 0.8981 - loss: 0.2377 - val_accuracy: 0.8935 - val_loss: 0.2562
Epoch 7/10
256/256 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - accuracy: 0.9035 - loss: 0.2307 - val_accuracy: 0.8897 - val_loss: 0.2590
Epoch 8/10
256/256 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step - accuracy: 0.9017 - loss: 0.2310 - val_accu

#Performance Metrics And Accuracy

In [64]:
from sklearn.metrics import confusion_matrix, accuracy_score
import pickle as pkl
y_pred = model.predict(X_test)
cm = confusion_matrix( y_test, y_pred.round())
print(cm)
accuracy = accuracy_score(y_test, y_pred.round())
print(accuracy)

172/172 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step
[[2672  439]
 [ 118 2257]]
0.8984688297484505
